# Examen Bimestral - Diseño de un Sistema Básico de Recuperación de Información

In [1]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd

## Requerimientos
### 1. Preprocesamiento de Datos

In [2]:
# Cargar el dataset de películas
movies = pd.read_csv('dataEx/rotten_tomatoes_movies.csv')
critic_review = pd.read_csv('dataEx/rotten_tomatoes_critic_reviews.csv')

# Visualizar las primeras filas para confirmar las columnas
# print(movies.head())
# print(critic_review.head())
print(movies[['movie_title', 'critics_consensus']].head())

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                   critics_consensus  
0  Though it may seem like just another Harry Pot...  
1  Nicole Holofcener's newest might seem slight i...  
2  Blake Edwards' bawdy comedy may not score a pe...  
3  Sidney Lumet's feature debut is a superbly wri...  
4  One of Disney's finest live-action adventures,...  


In [3]:
# Definir rutas de los archivos
ruta_criticas = "dataEx/rotten_tomatoes_critic_reviews.csv"
ruta_peliculas = "dataEx/rotten_tomatoes_movies.csv"

# Cargar los datos en DataFrames
criticas_df = pd.read_csv(ruta_criticas)
peliculas_df = pd.read_csv(ruta_peliculas)

# Normalizar tipos de datos para columnas clave
peliculas_df = peliculas_df.astype({
    "rotten_tomatoes_link": "string",
    "movie_title": "string"
})

# Mostrar un resumen de las columnas y tipos de datos después de la limpieza
print(peliculas_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17712 entries, 0 to 17711
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   rotten_tomatoes_link              17712 non-null  string 
 1   movie_title                       17712 non-null  string 
 2   movie_info                        17391 non-null  object 
 3   critics_consensus                 9134 non-null   object 
 4   content_rating                    17712 non-null  object 
 5   genres                            17693 non-null  object 
 6   directors                         17518 non-null  object 
 7   authors                           16170 non-null  object 
 8   actors                            17360 non-null  object 
 9   original_release_date             16546 non-null  object 
 10  streaming_release_date            17328 non-null  object 
 11  runtime                           17398 non-null  float64
 12  prod

In [4]:
# Configuración inicial
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
puntuacion = set(string.punctuation)

In [5]:
# Función de preprocesamiento
def procesar_texto(texto):
    if not isinstance(texto, str):  # Verificar si el texto es una cadena
        return ""  # Devuelve un string vacío si no lo es

    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar números y caracteres especiales
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    texto = re.sub(r'\s+', ' ', texto.strip())  # Remover espacios adicionales

    # Quitar signos de puntuación
    texto = ''.join([char for char in texto if char not in puntuacion])

    # Tokenización
    tokens = word_tokenize(texto)

    # Remover stopwords
    tokens_filtrados = [palabra for palabra in tokens if palabra not in stop_words]

    # Lematización
    tokens_lematizados = [lemmatizer.lemmatize(palabra) for palabra in tokens_filtrados]

    # Reconstruir texto limpio
    texto_procesado = ' '.join(tokens_lematizados)

    return texto_procesado

In [6]:
# Cargar el archivo de críticas
ruta_criticas = "dataEx/rotten_tomatoes_critic_reviews.csv"
criticas_df = pd.read_csv(ruta_criticas)

# Asegurarse de que 'review_content' tenga valores válidos
criticas_df = criticas_df.dropna(subset=['review_content'])
criticas_df['review_content'] = criticas_df['review_content'].apply(lambda x: str(x) if not pd.isnull(x) else "")

# Aplicar el preprocesamiento al corpus
print("Iniciando preprocesamiento del corpus...")
criticas_df['contenido_procesado'] = criticas_df['review_content'].apply(procesar_texto)

# Verificar resultado
print("Preprocesamiento completado. Ejemplo de contenido procesado:")
print(criticas_df[['review_content', 'contenido_procesado']].head())

Iniciando preprocesamiento del corpus...
Preprocesamiento completado. Ejemplo de contenido procesado:
                                      review_content  \
0  A fantasy adventure that fuses Greek mythology...   
1  Uma Thurman as Medusa, the gorgon with a coiff...   
2  With a top-notch cast and dazzling special eff...   
3  Whether audiences will get behind The Lightnin...   
4  What's really lacking in The Lightning Thief i...   

                                 contenido_procesado  
0  fantasy adventure fuse greek mythology contemp...  
1  uma thurman medusa gorgon coiffure writhing sn...  
2  topnotch cast dazzling special effect tide tee...  
3  whether audience get behind lightning thief ha...  
4  whats really lacking lightning thief genuine s...  


In [7]:
print(f"Filas originales: {len(criticas_df)}")
print(f"Filas con contenido preprocesado no vacío: {criticas_df['contenido_procesado'].str.strip().replace('', None).dropna().count()}")

Filas originales: 1064211
Filas con contenido preprocesado no vacío: 1064118


In [8]:
pd.set_option('display.max_rows', 20)  # Cambia 20 por el número que desees
print(criticas_df[['review_content', 'contenido_procesado']])

                                            review_content  \
0        A fantasy adventure that fuses Greek mythology...   
1        Uma Thurman as Medusa, the gorgon with a coiff...   
2        With a top-notch cast and dazzling special eff...   
3        Whether audiences will get behind The Lightnin...   
4        What's really lacking in The Lightning Thief i...   
...                                                    ...   
1130008  A rousing reconstruction of the 1879 Battle of...   
1130013  Seen today, it's not only a startling indictme...   
1130014  A rousing visual spectacle that's a prequel of...   
1130015  A simple two-act story: Prelude to war, and th...   
1130016  Rides the line between being a pure artifact o...   

                                       contenido_procesado  
0        fantasy adventure fuse greek mythology contemp...  
1        uma thurman medusa gorgon coiffure writhing sn...  
2        topnotch cast dazzling special effect tide tee...  
3        wh

In [9]:
criticas_df = criticas_df[criticas_df['contenido_procesado'].str.strip() != '']
print(f"Filas después de limpiar datos vacíos: {len(criticas_df)}")

Filas después de limpiar datos vacíos: 1064118


In [10]:
criticas_df.to_csv("dataEx/criticas_preprocesadas.csv", index=False)
print("Corpus preprocesado exportado a 'criticas_preprocesadas.csv'")

Corpus preprocesado exportado a 'criticas_preprocesadas.csv'


### 2. Construcción del Sistema

In [11]:
%pip install TfidfVectorizer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement TfidfVectorizer (from versions: none)
ERROR: No matching distribution found for TfidfVectorizer

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
print(criticas_df.columns)

Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content',
       'contenido_procesado'],
      dtype='object')


In [15]:
ruta = "dataEx/criticas_preprocesadas.csv"
criticas_df = pd.read_csv(ruta)
print(criticas_df.head())

  rotten_tomatoes_link      critic_name  top_critic           publisher_name  \
0            m/0814255  Andrew L. Urban       False           Urban Cinefile   
1            m/0814255    Louise Keller       False           Urban Cinefile   
2            m/0814255              NaN       False      FILMINK (Australia)   
3            m/0814255     Ben McEachen       False  Sunday Mail (Australia)   
4            m/0814255      Ethan Alter        True       Hollywood Reporter   

  review_type review_score review_date  \
0       Fresh          NaN  2010-02-06   
1       Fresh          NaN  2010-02-06   
2       Fresh          NaN  2010-02-09   
3       Fresh        3.5/5  2010-02-09   
4      Rotten          NaN  2010-02-10   

                                      review_content  \
0  A fantasy adventure that fuses Greek mythology...   
1  Uma Thurman as Medusa, the gorgon with a coiff...   
2  With a top-notch cast and dazzling special eff...   
3  Whether audiences will get behind The L

In [16]:
# Cargar ambos datasets
ruta_criticas = "dataEx/criticas_preprocesadas.csv"
ruta_peliculas = "dataEx/rotten_tomatoes_movies.csv"
criticas_df = pd.read_csv(ruta_criticas)
peliculas_df = pd.read_csv(ruta_peliculas)

# Realizar el merge usando la columna común, por ejemplo, 'rotten_tomatoes_link'
criticas_df = pd.merge(criticas_df, peliculas_df[['rotten_tomatoes_link', 'movie_title']], 
                       on='rotten_tomatoes_link', how='left')

In [17]:
print(criticas_df[['movie_title', 'contenido_procesado']].head())
print(criticas_df['movie_title'].isnull().sum())

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1  Percy Jackson & the Olympians: The Lightning T...   
2  Percy Jackson & the Olympians: The Lightning T...   
3  Percy Jackson & the Olympians: The Lightning T...   
4  Percy Jackson & the Olympians: The Lightning T...   

                                 contenido_procesado  
0  fantasy adventure fuse greek mythology contemp...  
1  uma thurman medusa gorgon coiffure writhing sn...  
2  topnotch cast dazzling special effect tide tee...  
3  whether audience get behind lightning thief ha...  
4  whats really lacking lightning thief genuine s...  
102


In [18]:
criticas_df.to_csv("dataEx/criticas_actualizadas.csv", index=False)

In [19]:
print(criticas_df.columns)

Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content',
       'contenido_procesado', 'movie_title'],
      dtype='object')


In [21]:
peliculas_df = pd.read_csv("dataEx/rotten_tomatoes_movies.csv")

# Asegúrate de que la columna común existe
print(peliculas_df.columns)  # Confirmar si 'rotten_tomatoes_link' está presente

# Realizar el merge
criticas_df = pd.merge(
    criticas_df,
    peliculas_df[['rotten_tomatoes_link', 'movie_title']],
    on='rotten_tomatoes_link',
    how='left'
)

# Verificar si 'movie_title' fue agregada
print(criticas_df[['rotten_tomatoes_link', 'movie_title']].head())

Index(['rotten_tomatoes_link', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'],
      dtype='object')
  rotten_tomatoes_link                                        movie_title
0            m/0814255  Percy Jackson & the Olympians: The Lightning T...
1            m/0814255  Percy Jackson & the Olympians: The Lightning T...
2            m/0814255  Percy Jackson & the Olympians: The Lightning T...
3            m/0814255  Percy Jackson & the Olympians: The Lightning T...
4            m/0814255  Percy Jackson & the Olympians: The Lightning T...


In [22]:
print(criticas_df['movie_title'].isnull().sum())

102


In [23]:
criticas_df = criticas_df.dropna(subset=['movie_title'])
print(f"Filas después de eliminar valores nulos: {len(criticas_df)}")

Filas después de eliminar valores nulos: 1064016


In [24]:
criticas_df['movie_title'].fillna('Título desconocido', inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_18728\3916570288.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  criticas_df['movie_title'].fillna('Título desconocido', inplace=True)


In [26]:
# peliculas_df = pd.read_csv("dataEx/rotten_tomatoes_movies.csv")
# criticas_df = pd.merge(
#     criticas_df,
#     peliculas_df[['rotten_tomatoes_link', 'movie_title']],
#     on='rotten_tomatoes_link',
#     how='left'
# )

In [27]:
print(criticas_df.columns)

Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content',
       'contenido_procesado', 'movie_title_x', 'movie_title_y', 'movie_title'],
      dtype='object')


In [29]:
# peliculas_df = pd.read_csv("dataEx/rotten_tomatoes_movies.csv")

# # Asegúrate de que la columna común existe
# print(peliculas_df.columns)  # Confirmar si 'rotten_tomatoes_link' está presente

# # Realizar el merge
# criticas_df = pd.merge(
#     criticas_df,
#     peliculas_df[['rotten_tomatoes_link', 'movie_title']],
#     on='rotten_tomatoes_link',
#     how='left'
# )

# # Verificar si 'movie_title' fue agregada
# print(criticas_df[['rotten_tomatoes_link', 'movie_title']].head())

In [30]:
criticas_df.to_csv("dataEx/criticas_actualizadas.csv", index=False)

In [31]:
print(criticas_df['movie_title'].isnull().sum())

0


In [33]:
# Vectorizar el corpus utilizando TF-IDF
vectorizador = TfidfVectorizer()
tfidf_matrix = vectorizador.fit_transform(criticas_df['contenido_procesado'])

# Función para procesar y responder consultas
def responder_consulta(consulta):
    consulta_procesada = procesar_texto(consulta)
    consulta_vectorizada = vectorizador.transform([consulta_procesada])
    similitudes = cosine_similarity(consulta_vectorizada, tfidf_matrix).flatten()
    indices_ordenados = similitudes.argsort()[::-1]
    resultados = criticas_df.iloc[indices_ordenados][['movie_title', 'contenido_procesado']]
    resultados_filtrados = resultados[similitudes[indices_ordenados] > 0.1]
    return resultados_filtrados.head(10)

# Ejemplo de consultas
consulta_1 = "Películas sobre viajes espaciales."
consulta_2 = "Películas para ver en familia."

print("Resultados para consulta 1:")
print(responder_consulta(consulta_1))

print("\nResultados para consulta 2:")
print(responder_consulta(consulta_2))

Resultados para consulta 1:
                                    movie_title  \
975008                                 Thirteen   
515812                   The Man Without a Past   
117461                          American Psycho   
344768                                 Get Real   
338035                             Frozen River   
362298  Grave of the Fireflies (Hotaru no haka)   
134092                      Assassination Tango   
666149                      Remember the Titans   
666150                      Remember the Titans   
781837                              The Soloist   

                                      contenido_procesado  
975008  e un claro ejemplo de que e posible hacer pelí...  
515812  lo grandioso de el hombre sin pasado e que e u...  
117461  una de la más comentadas ingeniosas e intelige...  
344768  película inteligente que trata sobre amor sobr...  
338035  una muy buena historia sobre la supervivencia ...  
362298  um retrato sem concesses horror da guerra 

### 3. Simulación de Consultas: El sistema debe mostrar un listado de las películas más relevantes para cada consulta, ordenadas según su relevancia.

In [36]:
print(criticas_df.columns)

Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content',
       'contenido_procesado', 'movie_title_x', 'movie_title_y', 'movie_title'],
      dtype='object')


In [38]:
print(criticas_df[['movie_title_x', 'movie_title_y', 'movie_title']].head())

                                       movie_title_x  \
0  Percy Jackson & the Olympians: The Lightning T...   
1  Percy Jackson & the Olympians: The Lightning T...   
2  Percy Jackson & the Olympians: The Lightning T...   
3  Percy Jackson & the Olympians: The Lightning T...   
4  Percy Jackson & the Olympians: The Lightning T...   

                                       movie_title_y  \
0  Percy Jackson & the Olympians: The Lightning T...   
1  Percy Jackson & the Olympians: The Lightning T...   
2  Percy Jackson & the Olympians: The Lightning T...   
3  Percy Jackson & the Olympians: The Lightning T...   
4  Percy Jackson & the Olympians: The Lightning T...   

                                         movie_title  
0  Percy Jackson & the Olympians: The Lightning T...  
1  Percy Jackson & the Olympians: The Lightning T...  
2  Percy Jackson & the Olympians: The Lightning T...  
3  Percy Jackson & the Olympians: The Lightning T...  
4  Percy Jackson & the Olympians: The Lightning T..

In [39]:
criticas_df['movie_title'] = criticas_df['movie_title_y']

In [40]:
criticas_df = criticas_df.drop(columns=['movie_title_x', 'movie_title_y'])

In [41]:
print(criticas_df.columns)

Index(['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date', 'review_content',
       'contenido_procesado', 'movie_title'],
      dtype='object')


In [43]:
def responder_consulta(consulta):
    consulta_procesada = procesar_texto(consulta)
    consulta_vectorizada = vectorizador.transform([consulta_procesada])
    similitudes = cosine_similarity(consulta_vectorizada, tfidf_matrix).flatten()
    indices_ordenados = similitudes.argsort()[::-1]
    resultados = criticas_df.iloc[indices_ordenados][['movie_title', 'contenido_procesado']]
    resultados_filtrados = resultados[similitudes[indices_ordenados] > 0.1]
    resultados_filtrados['relevancia'] = similitudes[indices_ordenados][:len(resultados_filtrados)]
    return resultados_filtrados.head(10)

In [44]:
consulta_1 = "Películas sobre viajes espaciales."
consulta_2 = "Películas para ver en familia."

print("Resultados para consulta 1:")
resultados_1 = responder_consulta(consulta_1)
print(resultados_1[['movie_title', 'relevancia']])

print("\nResultados para consulta 2:")
resultados_2 = responder_consulta(consulta_2)
print(resultados_2[['movie_title', 'relevancia']])

Resultados para consulta 1:


C:\Users\USER\AppData\Local\Temp\ipykernel_18728\945678219.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados_filtrados['relevancia'] = similitudes[indices_ordenados][:len(resultados_filtrados)]


                                    movie_title  relevancia
975008                                 Thirteen    0.287986
515812                   The Man Without a Past    0.273198
117461                          American Psycho    0.273189
344768                                 Get Real    0.255205
338035                             Frozen River    0.244823
362298  Grave of the Fireflies (Hotaru no haka)    0.239448
134092                      Assassination Tango    0.234337
666149                      Remember the Titans    0.230089
666150                      Remember the Titans    0.230089
781837                              The Soloist    0.223256

Resultados para consulta 2:
                               movie_title  relevancia
82977                       50 First Dates    0.295523
241977                         Coyote Ugly    0.288579
487436           Life or Something Like It    0.264361
441861          Jimmy Neutron - Boy Genius    0.258626
939498                             T

C:\Users\USER\AppData\Local\Temp\ipykernel_18728\945678219.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados_filtrados['relevancia'] = similitudes[indices_ordenados][:len(resultados_filtrados)]


### 4. Análisis de Resultados